In [ ]:
import os
from mytool import json2label, normalize_boxes, make_text_label



##### 전역변수 선언

In [18]:
import os

DATASET_DIR_PATH = "./dataset"
SAVE_IMAGES_DIR_PATH = os.path.join(DATASET_DIR_PATH, "images")
SAVE_LABELS_DIR_PATH = os.path.join(DATASET_DIR_PATH, "labels")
BG_IMG_DIR_PATH = "./seed_data/conveyor_img"
OBG_IMG_DIR_PATH = "./seed_data/product_imgs"
OBG_JSON_DIR_PATH = "./seed_data/product_json"

NAME_TO_ID = {"obj":0, "date":1}

In [8]:
# 폴더 없으면 만들기
import os

for path in [DATASET_DIR_PATH, SAVE_IMAGES_DIR_PATH, SAVE_LABELS_DIR_PATH]:
    if not os.path.exists(path):
        os.makedirs(path)

##### 이미지 증축

In [16]:
import cv2
import random as rd
def random_resize(product_img, img_size, boxes): # 0.7 ~ 1.5
    alpha = 0.8*rd.random() + 0.7
    new_size = np.array(img_size) * alpha
    new_boxes = boxes * alpha
    new_img = cv2.resize(product_img, new_size)
    return new_img, new_size, new_boxes

import random as rd
def random_point(bg_size, img_size):
    range_x = bg_size[0] - img_size[0]
    range_y = bg_size[1] - img_size[1]
    return rd.randint(0, range_x), rd.randint(0, range_y)

def synthetic_img(product_img, bg_img, img_size, r_xy):
    r_x, r_y = r_xy
    x, y = img_size
    assert r_x+x <= bg_img.shape[1] and r_y+y <= bg_img.shape[0], "합성범위 넘어감"
    bg_img[r_y:r_y+y, r_x:r_x+x] = product_img
    return bg_img

import numpy as np
from mytool import normalize_boxes
def gen_data_one(product_img, product_label, bg_img): # return img, shape(n,5)
    names, boxes, imgsize = product_label # 제품과 날짜 box
    assert product_img.shape[::-1] == imgsize
    
    bg_size = bg_img.shape[::-1]
    product_img, img_size, boxes = random_resize(product_img, img_size, boxes) # 제품이미지와 박스 사이즈 변경
    r_xy = random_point(bg_size, img_size) # 배경이미지안의 랜덤좌표
    assert type(r_xy) == tuple
    new_img = synthetic_img(product_img, bg_img, img_size, r_xy) # 이미지합성
    boxes += r_xy*2 # (a,b,a,b) # 이동한 만큼 박스 좌표도 변경

    ids = np.array([NAME_TO_ID[name] for name in names])[:,None]
    boxes = normalize_boxes(bg_size, boxes) # ndarray
    assert len(ids)==len(boxes)
    label = np.concatenate([ids, boxes], axis=-1)
    return new_img, label
    

import cv2
from glob import glob
import random as rd
from mytool import json2label, make_text_label
def gen_data():
    bg_img_path = glob(BG_IMG_DIR_PATH + "/*.jpg")[0]
    bg_img = cv2.imread(bg_img_path)
    product_img_paths = sorted(glob(OBG_IMG_DIR_PATH + "/*.jpg"))
    product_json_paths = sorted(glob(OBG_JSON_DIR_PATH + "/*.json"))
    product_imgs = [cv2.imread(path) for path in product_img_paths]
    product_labels = [json2label(path) for path in product_json_paths]
    for i in range(1000):
        v = rd.choice(range(len(product_imgs)))
        img, label = gen_data_one(product_imgs[v], product_labels[v], bg_img) # img, shape(n, 5) # 0 0.123 0.234 0.345 0.456
        cv2.imwrite(SAVE_IMAGES_DIR_PATH + "/" + f"{i:04d}.jpg", img)
        make_text_label(label[:,0], label[:, 1:], SAVE_LABELS_DIR_PATH + "/" + f"{i:04d}.txt")